## dependencies

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pickle
import gc

## creating the dataset

In [2]:
path = '../input/amazonsatelliteimages/train-jpg'

for img in os.listdir(path):
    img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
    plt.imshow(img_arr)
    # printing one image
    break

In [7]:
gc.collect()
IMG_SIZE = 50

train_data = []

def create_train_data():

    for img in os.listdir(path):
      try:
        img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        new_img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        train_data.append(new_img_arr)
      except Exception as e:
        print(e)
        
create_train_data()

In [20]:
labels = pd.read_csv('../input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')

X = np.array(train_data)
y = np.array(labels['tags'])

pickle.dump(X, open('./X.pkl','wb'))
pickle.dump(y, open('./y.pkl','wb'))